# Xarray-spatial
### User Guide: Remote Sensing tools
-----

Xarray-spatial's Remote Sensing tools provide a range of functions pertaining to remote sensing data such as satellite imagery. A range of functions are available to calculate various vegetation and environmental parameters from the range of band data available for an area. These functions accept and output data in the form of xarray.DataArray rasters.

[True Color](#True-Color)   
[Vegetation Index](#Vegetation-Index): [NDVI](#NDVI), [SAVI](#SAVI), [ARVI](#ARVI), [EVI](#EVI)   
[Green Chlorophyll Index - GCI](#Green-Chlorophyll-Index-(GCI))  
[Normalized Burn Ratio](#Normalized-Burn-Ratio): [NBR](#NBR), [NBR2](#NBR2)    
[Normalized Difference Moisture Index - NDMI](#Normalized-Difference-Moisture-Index-(NDMI))  
[Structure Insensitive Pigment Index - SIPI](#Structure-Insensitive-Pigment-Index-(SIPI))  
[Enhanced Built-Up and Bareness Index - EBBI](#Enhanced-Built-Up-and-Bareness-Index-(EBBI))  
[Bump Mapping](#Bump-Mapping)  

-----------


## Load data

To get started, we'll import some basic packages, along with several handy datashader functions, mainly for rendering.

To download the examples data, run the command `xrspatial examples` in your terminal. All the data will be stored in your current directory inside a folder named `xrspatial-examples`.

In [ ]:
import datashader as ds
from datashader.colors import Elevation
import datashader.transfer_functions as tf
from datashader.transfer_functions import shade
from datashader.transfer_functions import stack
from datashader.transfer_functions import dynspread
from datashader.transfer_functions import set_background
from datashader.transfer_functions import Images, Image
from datashader.utils import orient_array
import numpy as np
import xarray as xr

The following functions apply to image data with bands in different parts of the UV/Visible/IR spectrum (multispectral), so we'll bring in some multispectral satellite image data to work with.

Below, we loaded all of the images and transformed them into the form of an xarray DataArray to use in the Xarray-spatial functions.

In [ ]:
SCENE_ID = "LC80030172015001LGN00"
EXTS = {
    "coastal_aerosol": "B1",
    "blue": "B2",
    "green": "B3",
    "red": "B4",
    "nir": "B5",
    "swir1": "B6",
    "swir2": "B7",
    "panchromatic": "B8",
    "cirrus": "B9",
    "tir1": "B10",
    "tir2": "B11",
    "qa": "BQA",
}

cvs = ds.Canvas(plot_width=1024, plot_height=1024)
layers = {}
for name, ext in EXTS.items():
    layer = xr.open_rasterio(
        f"../../xrspatial-examples/data/{SCENE_ID}_{ext}.tiff"
    ).load()[0]
    layer.name = name
    layer = cvs.raster(layer, agg="mean")
    layer.data = orient_array(layer)
    layers[name] = layer
layers

##### Let's do a quick visualization to see what these images look like. 

In [ ]:
shaded = []
for name, raster in layers.items():
    img = shade(raster)
    img.name = name
    shaded.append(img)

imgs = Images(*shaded)
imgs.num_cols = 2
imgs

## True Color

Now we're ready to apply some xarray-spatial functions. 

To start, we can apply `true_color` to the red, green, and blue bands from above to generate a real-looking image.

In [ ]:
from xrspatial.multispectral import true_color

true_color_agg = true_color(layers["red"], layers["green"], layers["blue"])
Image(true_color_agg)

## Vegetation Index

### NDVI

The [Normalized Difference Vegetation Index](https://en.wikipedia.org/wiki/Normalized_difference_vegetation_index) (NDVI) is a metric designed to detect regions with vegetation by measuring the difference between near-infrared (NIR) light (which vegetation reflects) and red light (which vegetation absorbs).

The NDVI ranges over [-1,+1], where `-1` means more "Red" radiation while `+1` means more "NIR" radiation. NDVI values close to +1.0 suggest areas dense with active green foliage, while strongly negative values suggest cloud cover or snow, and values near zero suggest open water, urban areas, or bare soil.

Let's apply `xrspatial.multispectral.ndvi` to the satellite band images from above.

In [ ]:
from xrspatial.multispectral import ndvi

ndvi_agg = ndvi(nir_agg=layers["nir"], red_agg=layers["red"])

### SAVI

`xrspatial.multispectral.savi` also computes the vegetation index from the red and nir bands, but it applies a correction factor for the soil brightness.

Let's try applying that to our bands from above.

In [ ]:
from xrspatial.multispectral import savi

savi_agg = savi(layers["nir"], layers["red"])

### ARVI

`xrspatial.multispectral.arvi` computes the Atmospherically Resistant vegetation index from the blue, red and nir bands. It applies molecular and ozone correction with no further need for aerosol correction, except for dust conditions.

Let's try applying that to our bands data.

In [ ]:
from xrspatial.multispectral import arvi

arvi_agg = arvi(layers["nir"], layers["red"], layers["blue"])

### EVI

`xrspatial.multispectral.evi` computes the Enhanced Vegetation Index. It allows for importved sensitivity in high biomass regions, de-coupling of the canopy background signal and reduction of atmospheric influences. The function uses nir, red and blue bands.

Let's try applying that to our bands data.

In [ ]:
from xrspatial.multispectral import evi

evi_agg = evi(layers["nir"], layers["red"], layers["blue"])

### Visualize all vegetation indexes

In [ ]:
vegetation_index_aggs = [ndvi_agg, savi_agg, arvi_agg, evi_agg]
images = [shade(agg, cmap=['purple', 'black', 'green']) for agg in vegetation_index_aggs]

Images(*images)

## Green Chlorophyll Index (GCI)

`xrspatial.gci` computes the Green Chlorophyll Index. It can be used to estimate the content of leaf chorophyll and predict the physiological state of vegetation and plant health. The function uses nir and green band data.

Let's apply that to our bands data.

In [ ]:
from xrspatial.multispectral import gci

gci_agg = gci(layers["nir"], layers["green"])
shade(gci_agg)

## Normalized Burn Ratio

`xrspatial.multispectral` provides 2 tools for computing the Normalized Burn Ratio. `xrspatial.multispectral.nbr` is to identify burned areas and provide a measure of burn severity while `xrspatial.multispectral.nbr2` modifies the Normalized Burn Ratio (NBR) to highlight water sensitivity in vegetation and may be useful in post-fire recovery studies.


### NBR
`xrspatial.multispectral.nbr` uses NIR and SWIR2 band data. Let's calculate NBR using the our example data.

In [ ]:
from xrspatial.multispectral import nbr

nbr_agg = nbr(layers["nir"], layers["swir2"])
shade(nbr_agg)

### NBR2
`xrspatial.multispectral.nbr2` uses SWIR1 and SWIR2 band data. Let's calculate NBR2 using the our example data.

In [ ]:
from xrspatial.multispectral import nbr2

nbr2_agg = nbr2(layers["swir1"], layers["swir2"])
shade(nbr2_agg)

## Normalized Difference Moisture Index (NDMI)  

`xrspatial.multispectral.ndmi` calculates the Normalized Difference Moisture Index. It can be used to determine vegetation water content. The function uses NIR and SWIR1 band data.

In [ ]:
from xrspatial.multispectral import ndmi

ndmi_agg = ndmi(layers["nir"], layers["swir1"])
shade(ndmi_agg)

## Structure Insensitive Pigment Index (SIPI)  

`xrspatial.multispectral.sipi` calculates the Structure Insensitive Pigment Index which helpful in early disease detection in vegetation. The function uses NIR, red and blue band data.

In [ ]:
from xrspatial.multispectral import sipi

sipi_agg = sipi(layers["nir"], layers["red"], layers["blue"])
shade(sipi_agg)

## Enhanced Built-Up and Bareness Index (EBBI)

`xrspatial.multispectral.sipi` computes the Enhanced Built-Up and Bareness Index (EBBI) which allows for easily distinguishing between built-up and bare land areas. The function uses red, SWIR, and TIR band data.

In [ ]:
from xrspatial.multispectral import ebbi

ebbi_agg = ebbi(layers["red"], layers["swir1"], layers["tir1"])
shade(ebbi_agg)

## Bump Mapping

For the next few functions, we'll experiment with an artificial terrain. We'll use xarray-spatial's `generate_terrain` along with datashader's Canvas to smooth thing.

### Generate Terrain

In [ ]:
from xrspatial import generate_terrain

W = 800
H = 600

template_terrain = xr.DataArray(np.zeros((H, W)))
x_range=(-20e6, 20e6)
y_range=(20e6, -20e6)

terrain = generate_terrain(template_terrain, x_range=x_range, y_range=y_range)
terrain.attrs["unit"] = "meter"

shade(terrain, cmap=["black", "white"], how="linear")

The grayscale values in the image above show the elevation, scaled linearly in intensity (with the large black areas indicating low elevation). This is good, but it would look more like a landscape if we map the lowest values to colors representing water, and the highest to colors representing mountaintops. We can use the Elevation colormap for this.

In [ ]:
shade(terrain, cmap=Elevation, how="linear")

Now we can generate the rgba PIL image, extract the green and blue bands, and input those into ndvi. 

The result is displayed below. 

In [ ]:
rgba = stack(shade(terrain, cmap=Elevation, how="linear")).to_pil()
r, g, b, a = [
    xr.DataArray(np.flipud(np.asarray(rgba.getchannel(c)))) / 255.0
    for c in ["R", "G", "B", "A"]
]

ndvi_img = ndvi(nir_agg=g, red_agg=b)
shade(ndvi_img, cmap=["purple", "black", "green"], how="linear")

### Bump

Bump mapping is a cartographic technique that can be used to create the appearance of trees or other land features, which is useful when synthesizing human-interpretable images from source data like land use classifications.

`xrspatial.bump` will produce a bump aggregate for adding detail to the terrain.

In this example, we will pretend the bumps are trees, and shade them with green.  We'll also use the elevation data to modulate whether there are trees and if so how tall they are.

- First, we'll define a custom `height` function to return tree heights suitable for the given elevation range
- `xrspatial.bump` accepts a function with only a single argument (`locations`), so we will use `functools.partial` to provide values for the other arguments.
- Bump mapping isn't normally a performance bottleneck, but if you want, you can speed it up by using Numba on your custom `height` function (`from xrspatial.utils import ngjit`, then put `@ngjit` above `def heights(...)`).

In [ ]:
from functools import partial

from xrspatial import bump, hillshade


def heights(locations, src, src_range, height=20):
    num_bumps = locations.shape[0]
    out = np.zeros(num_bumps, dtype=np.uint16)
    for r in range(0, num_bumps):
        loc = locations[r]
        x = loc[0]
        y = loc[1]
        val = src[y, x]
        if val >= src_range[0] and val < src_range[1]:
            out[r] = height
    return out


T = 300000  # Number of trees to add per call
src = terrain.data
%time trees  = bump(W, H, count=T,    height_func=partial(heights, src=src, src_range=(1000, 1300), height=5))
trees += bump(
    W,
    H,
    count=T // 2,
    height_func=partial(heights, src=src, src_range=(1300, 1700), height=20),
)
trees += bump(
    W,
    H,
    count=T // 3,
    height_func=partial(heights, src=src, src_range=(1700, 2000), height=5),
)
trees = trees.assign_coords(terrain.coords)

tree_colorize = trees.copy()
tree_colorize.data[tree_colorize.data == 0] = np.nan
hillshaded = hillshade(terrain + trees)

stack(
    shade(terrain, cmap=["black", "white"], how="linear"),
    shade(hillshaded, cmap=["black", "white"], how="linear", alpha=128),
    shade(tree_colorize, cmap="limegreen", how="linear"),
)